In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train_df = pd.read_csv("/kaggle/input/nyse/prices-split-adjusted.csv")
print(train_df.head(5))

In [10]:
train_df = pd.read_csv("/kaggle/input/nyse/prices-split-adjusted.csv")
print(train_df.head(5))

In [93]:
train_df_2 = pd.read_csv("/kaggle/input/nyse/prices-split-adjusted.csv")

In [109]:
train_df_2.groupby('symbol').count().nlargest(15,'date')

In [12]:
len(set(train_df.symbol))

There is data of 501 companies, we can choose one of them say WLTW and visualize them.

In [13]:
train_df = train_df[train_df['symbol']=='A']


In [14]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.plot(range(train_df.shape[0]), (train_df['open']+train_df['close'])/2)
plt.xticks(range(0,train_df.shape[0],500), train_df['date'].loc[::500])
plt.xlabel("Date")
plt.ylabel("Stock prices")
plt.title("Stock market variation over years")
plt.show()

The prices towards the later years are more and have more fluctuations, so it is important to normalize the data. Normalization is done in windows.

In [ ]:
print(mid_prices[:10])

In [15]:
mid_prices = (train_df.loc[:,'high']+train_df.loc[:,'low'])/2
train = mid_prices[:1500]
test = mid_prices[1500:]

In [ ]:
train[0:10].head(3)

In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
window_size = 2500

train = train.values.reshape(-1, 1)
test = test.values.reshape(-1, 1)

In [ ]:
print(len(train))

In [17]:
window_size =20
for i in range(0, 1480, window_size):
    scaler.fit(train[i:i+window_size])
    train[i:i+window_size] = scaler.transform(train[i:i+window_size])
    
scaler.fit(train[i+window_size:])
train[i+window_size:] = scaler.transform(train[i+window_size:])
test = scaler.transform(test)

In [18]:
train = train.reshape(-1)
test = test.reshape(-1)

can make data smoother using exponential moving average

In [19]:
ema = 0.0
gamma = 0.1
for i in range(220):
    ema = gamma*train[i] + (1-gamma)*ema
    train[i] = ema
    
all_data = np.concatenate([train, test], axis=0)

In [ ]:
N

Will try out 2 averaging methods, standard moving average and exponential

In [20]:
window_size = 10
N = train.size
preds = []
errors = []

for idx in range(window_size, N):
    preds.append(np.mean(train[idx-window_size: idx]))
    errors.append((preds[-1]-train[idx])**2)
    
print("MSE is ", np.mean(errors))

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(50.7,8.27)})
sns.lineplot(range(0,1500),all_data[:1500], color='blue', label='true')
ax = sns.lineplot(range(window_size, N), preds, color='red',label='prediction')
ax.set(xlabel='date', ylabel='mid-price')
plt.show()

In [23]:
running_mean = 0.0
preds = []
errors = []
preds.append(running_mean)
decay = 0.5
for idx in range(1,N):
    running_mean = running_mean*decay + (1-decay)*train[idx-1]
    preds.append(running_mean)
    errors.append((preds[-1]-train[idx])**2)
print("MSE is ", np.mean(errors))

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(50.7,8.27)})
sns.lineplot(range(0,1500),all_data[:1500], color='blue', label='true')
ax = sns.lineplot(range(0, N), preds, color='red',label='prediction')
ax.set(xlabel='date', ylabel='mid-price')
plt.show()

But all these averaging are helpful to predict for next day, not for cases like next month or after 30 days. We need to model in such a way that we predict into the future. We create a pattern set, that is price of few days as teh feature to predict 6th day.

In [25]:
def createPattern(data, step=5):
    pattern = []
    price = []
    for day in range(step, data.shape[0]):
        pattern.append(data[day-step:day])
        price.append(data[day])
    pattern, price = np.array(pattern), np.array(price)
    pattern = pattern.reshape(pattern.shape[0], pattern.shape[1], 1)
    return pattern, price
        

In [78]:
train_pattern = createPattern(train, 60)
X_train = train_pattern[0]
y_train = train_pattern[1]

In [79]:
test_pattern = createPattern(test, 60)
X_test = test_pattern[0]
y_test = test_pattern[1]

In [28]:
import tensorflow as tf
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Activation, SimpleRNN, Dense, LSTM, Dropout

In [80]:
class RNN():
    
    neurons = 60
    def __init__(self, X_train, y_train, epoch):
        self.X_train = X_train
        self.y_train = y_train
        self.epoch = epoch
        self.batch_size = 32
        self.loss = tf.keras.metrics.mean_squared_error
        self.neurons = 60
        
    def buildArch(self, num_rnn, num_dense):
        self.model = tf.keras.Sequential()
        self.model.add(SimpleRNN(self.neurons, activation='tanh',return_sequences = True,input_shape = (self.X_train.shape[1],1)))
        self.model.add(Dropout(0.2))
        for i in range(num_rnn):
            self.model.add(SimpleRNN(self.neurons, activation='tanh',return_sequences = True,input_shape = (self.X_train.shape[1],1)))
            self.model.add(Dropout(0.2))     
        self.model.add(SimpleRNN(self.neurons, activation='tanh',return_sequences = False,input_shape = (self.X_train.shape[1],1)))
        for i in range(num_dense):
            self.model.add(Dense(self.neurons, activation='tanh'))
        self.model.add(Dense(1))
        return self.model.summary()
    
    def compile(self):
        opt = tf.keras.optimizers.Adam()
        self.model.compile(optimizer= opt, loss= self.loss)
        return self.model.summary
    
    def modelFit(self):
        history = self.model.fit(self.X_train, self.y_train, epochs = self.epoch, batch_size = self.batch_size, validation_split=0.2)
        return history

In [81]:
rnn = RNN(X_train, y_train, 60)
rnn.buildArch(2,1)
rnn.compile()
print(X_train.shape)

In [82]:
history = rnn.modelFit()

In [83]:
pred = rnn.model.predict(X_test)

In [84]:
pred = pred.reshape(-1)

In [ ]:
len(X_test)

In [85]:
sns.set(rc={'figure.figsize':(50.7,8.27)})
sns.lineplot(range(0,len(X_test)),y_test, color='blue', label='true')
ax = sns.lineplot(range(0,len(X_test)), pred, color='red',label='prediction')
ax.set(xlabel='date', ylabel='mid-price')
plt.show()

In [86]:
class LSTM(RNN):
    RNN.model = tf.keras.Sequential()
    
    def __init__(self, X_train, y_train, epochs):
        super().__init__(X_train, y_train, epochs)
        
    def buildArch(self, num_dense = 1):
        RNN.model = tf.keras.Sequential()
        RNN.model.add(tf.keras.layers.LSTM(10,input_shape=(None,1)))
        RNN.model.add(Dense(units=1))
        return RNN.model.summary()
        

In [87]:
lstm = LSTM(X_train, y_train, 60)
lstm.buildArch(1)
lstm.compile()


In [89]:
history = lstm.modelFit()

In [90]:
pred = lstm.model.predict(X_test)
pred = pred.reshape(-1)

In [91]:
sns.set(rc={'figure.figsize':(50.7,8.27)})
sns.lineplot(range(0,len(X_test)),y_test, color='blue', label='true')
ax = sns.lineplot(range(0,len(X_test)), pred, color='red',label='prediction')
ax.set(xlabel='date', ylabel='mid-price')
plt.show()

get stock for different companies and see relation

Can compare for a given industry i.e. companies in a single industry

In [110]:
returns = pd.DataFrame()

tickers = ['A','AAL','AAP','AAPL','ABC']
for tick in tickers:
    returns[tick+'_return']=train_df_2[train_df_2['symbol']=='A']['close']
returns.head(5)

In [97]:
import seaborn as sns
sns.pairplot(returns[1:])

In [98]:
print(returns.std())

Whichever has more std deviation is more riskier

In [102]:
print(returns.head(3))

In [106]:
sns.heatmap(returns.corr())

In [120]:
train_df_2[train_df_2['symbol']=='A'][['close']].iplot(kind='candle')